In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
from tqdm import tqdm

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification')
sys.path.append('/content/drive/MyDrive/nlp_ss24/multilingual-lexical-simplification/src')

In [4]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("lusxvr/distilbert-base-multilingual-cased-finetuned-leiko")
model = AutoModelForMaskedLM.from_pretrained("lusxvr/distilbert-base-multilingual-cased-finetuned-leiko")

# Specify the patter to use for this lexical simplifier
bert_pattern = '{original_sentence} Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/598 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

In [5]:
from simple_bert_lexical_simplifier import SimpleBertLexicalSimplifier
bert_ls = SimpleBertLexicalSimplifier(model, tokenizer, bert_pattern, None)

Using cuda for model inference.
No exemplars provided, using zero-shot mode.


In [6]:
from benchmark_suite import BenchmarkSuite
from language import Language
suite = BenchmarkSuite(bert_ls, {Language.EN: '{original_sentence} The simpler version of the previous sentence is: {sentence_with_complex_word_masked}',
                                 Language.DE: '{original_sentence} Die vereinfachte Version des vorigen Satzes ist: {sentence_with_complex_word_masked}'})
suite.run()

Benchmarking model on EN ...
Benchmarking model on BenchLSDataProvider...


Benchmarking: 100%|██████████| 929/929 [00:16<00:00, 57.64it/s]


Benchmarking model on LexMTurkDataProvider...


Benchmarking: 100%|██████████| 500/500 [00:05<00:00, 91.38it/s]


Benchmarking model on NNSevalDataProvider...


Benchmarking: 100%|██████████| 239/239 [00:02<00:00, 83.52it/s]


Benchmarking model on TsarENDataProvider...


Benchmarking: 100%|██████████| 386/386 [00:04<00:00, 77.91it/s]


Benchmarking model on DE ...
Benchmarking model on GermanEvalDataProvider...


Benchmarking: 100%|██████████| 1040/1040 [00:12<00:00, 82.68it/s]
